## 计算图的加法和乘法层的实现

In [3]:
# 计算图的乘法层
class MulLayer():
    
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = self.x * self.y
        
        return out
    
    def backward(self, dout):
        #乘法的反向是交换x y
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy


In [2]:
# 测试乘法层 用书上苹果的案例 
# y = x1 * x2 * x3
x1 = 100  # 苹果单价
x2 = 2    # 苹果个数
x3 = 1.1  # 0.1的消费税


# 第一层 苹果单价*个数 
# 第二层 第一层的输出*税

layer1 = MulLayer()
layer2 = MulLayer()

# forward
o1 = layer1.forward(x1, x2)
o2 = layer2.forward(o1, x3)
print(f'{o2}')
# o2就是x1 * x2 * x3

# backward
d = 1.0
do1, d3 = layer2.backward(d)
d1, d2 = layer1.backward(do1)
print(d1, d2, d3)

# d1/d2/d3表示x1/x2/x3增大1的话，输出y增大多少


220.00000000000003
2.2 110.00000000000001 200.0


In [29]:
# 计算图的加法层
# 加法层比较简单，由于反向是把d直接传递出去，因此不需要存储x和y
class AddLayer():
    
    def __init__(self):
        pass
    
    def forward(self, x, y):
        return x + y
    
    def backward(self, dout):
        return dout, dout


In [30]:
# 测试加法层, 用书上苹果和橘子的案例
# (苹果个数*苹果单价 + 橘子个数*橘子单价)*消费税=最终价

x1 = 100  # 苹果单价
x2 = 2    # 苹果个数
x3 = 150  # 橘子单价
x4 = 3    # 橘子个数
x5 = 1.1  # 税

layer1 = MulLayer() # 计算苹果总价
layer2 = MulLayer() # 计算橘子总价
layer3 = AddLayer() # 计算苹果+橘子总价
layer4 = MulLayer() # 计算税后价格，即最终价格

# forward 计算总价
o1 = layer1.forward(x1, x2)
o2 = layer2.forward(x3, x4)
o3 = layer3.forward(o1, o2)
o4 = layer4.forward(o3, x5)
print(o4)

# backward 计算x1-x5每提升1，对输出的影响
d = 1
do4, d5 = layer4.backward(d)
do31, do32 = layer3.backward(do4)
d1, d2 = layer1.backward(do31)
d3, d4 = layer2.backward(do32)
print(d1, d2, d3, d4, d5)

# 这里的5个输出应该对照书里的计算图(书138页/pdf163页)上的反向数值一致


715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0 650
